<a href="https://colab.research.google.com/github/AeroEng16/FootballAnalysis/blob/main/inferenceTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import re
import json
import datetime
import pandas as pd
from PIL import Image, ImageDraw
from io import BytesIO
import plotly.express as px
import yaml
import os
try:
    from roboflow import Roboflow
except ImportError as e:
    !pip install roboflow

try:
  from ultralytics import YOLO
except ImportError as e:
  !pip install ultralytics
  from ultralytics import YOLO
from itertools import product

;

''

In [ ]:
robowFlowAPIKey ="e4YWipTNVce3CfUmX7fc"

In [28]:

imageURL = "https://raw.githubusercontent.com/AeroEng16/FootballAnalysis/main/oldFirmTestImage.png?raw=true"
imageURL

response = requests.get(imageURL)
img = Image.open(BytesIO(response.content))
img1 = ImageDraw.Draw(img)


#display(img)

In [9]:
!wget https://raw.githubusercontent.com/AeroEng16/FootballAnalysis/main/bestFootballYoloSoFar.pt


--2024-08-22 11:59:42--  https://raw.githubusercontent.com/AeroEng16/FootballAnalysis/main/bestFootballYoloSoFar.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5760691 (5.5M) [application/octet-stream]
Saving to: ‘bestFootballYoloSoFar.pt’

bestFootballYoloSoF 100%[===================>]   5.49M  --.-KB/s    in 0.06s   

2024-08-22 11:59:43 (85.6 MB/s) - ‘bestFootballYoloSoFar.pt’ saved [5760691/5760691]



In [8]:
def tile(imageURL, dir_out, d):
    response = requests.get(imageURL)
    img = Image.open(BytesIO(response.content))
    w, h = img.size

    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        out = os.path.join(dir_out, f'{"test"}_{i}_{j}{".png"}')
        img.crop(box).save(out)

tile(imageURL,"/content/subimages",640)

In [41]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO("bestFootballYoloSoFar.pt")

# Define path to the image file
source = "/content/subimages/test_0_0.png"

# Run inference on the source
resultsPartialImage = model(source)  # list of Results objects
resultsFullImage = model(img)  # list of Results objects


image 1/1 /content/subimages/test_0_0.png: 640x640 1 referee, 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 416x640 13 players, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)


In [39]:
resultsFullImage[0].boxes.xyxy.cpu().numpy()[1]

array([     256.45,      482.74,      287.46,      615.95], dtype=float32)

In [43]:
imgPartial = Image.open(source)
fig = px.imshow(imgPartial)
for r in resultsPartialImage[0].boxes.xyxy.cpu().numpy():
    currentBox= r
    print(currentBox)
    fig.add_shape(
        type='rect',
        x0=currentBox[0], x1=currentBox[2], y0=currentBox[1], y1=currentBox[3],
        xref='x', yref='y',
        line_color='cyan'
    )

fig.show()

[     267.87      454.03      294.28      542.08]


In [44]:

fig = px.imshow(img)

for r in resultsFullImage[0].boxes.xyxy.cpu().numpy():
    currentBox= r
    print(currentBox)
    fig.add_shape(
        type='rect',
        x0=currentBox[0], x1=currentBox[2], y0=currentBox[1], y1=currentBox[3],
        xref='x', yref='y',
        line_color='cyan'
    )

fig.show()

[     299.36      639.16       349.5      792.93]
[     256.45      482.74      287.46      615.95]
[     1102.8      507.19      1137.6      650.49]
[     1105.2      271.58      1144.3      368.39]
[     14.598      544.96      60.482      694.13]
[     616.95      311.73      647.17      422.57]
[     301.98      384.18      335.39      498.24]
[     918.73      764.91      960.19       923.6]
[     963.04      303.96      999.47      392.39]
[     978.61      318.49      1009.4      404.61]
[     1121.7       421.3        1155      546.06]
[     40.303      544.67      77.362      690.93]
[     286.32      388.27      326.78      496.28]
